## Project to create two maps - marker and cluster_marker, showing each bike rack in Davidson County.

In [1]:
from shapely.geometry import Point   #This is to pull Point module (could import ,Line after Point if wanted)
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

In [2]:
racks = gpd.read_file('../data/Bike Racks (GIS).geojson')  #Using gpd geopandas to read in the file
print(racks.crs)
racks.head( )

{'init': 'epsg:4326'}


,detail_loc,quantity,location,domain,idnumber,objectid,status,lon,type,data_colle,lat,capacity,geometry
0,Hickory Hollow Mall,1,Mall,Public,113,1,Existing,-86.6554527779,Rack,Health Dept,36.0499277778,12,POINT (-86.65545 36.04993)
1,Stones River Greenway - Donelson/Hermitage Area,2,Greenway,Public,114,2,Existing,-86.6340625226,Rack,Health Dept,36.1852800378,4,POINT (-86.63406 36.18528)
2,"Percy Priest Reservoir, Stones River Greenway",0,Park/Greenway,Public,256,3,Proposed,-86.6206705697,Rack,Health Dept,36.1584215986,0,POINT (-86.62067 36.15842)
3,"Two Rivers Golf Course, Stones River Greenway ...",1,Park/Greenway,Public,115,4,Existing,-86.6865259267,Rack,Health Dept,36.1875466948,5,POINT (-86.68653 36.18755)
4,East Park & Community Center,3,Park/Community Center,Public,103,5,Existing,-86.7621800831,Rack,Health Dept,36.1724554062,15,POINT (-86.76218 36.17246)


In [3]:
#To convert type to proper name EPSG:4326
racks.crs = "EPSG:4326"
print(racks.crs)

EPSG:4326


In [4]:
racks.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 430 entries, 0 to 429
Data columns (total 13 columns):
detail_loc    430 non-null object
quantity      430 non-null object
location      430 non-null object
domain        430 non-null object
idnumber      430 non-null object
objectid      430 non-null object
status        430 non-null object
lon           426 non-null object
type          430 non-null object
data_colle    430 non-null object
lat           426 non-null object
capacity      430 non-null object
geometry      430 non-null geometry
dtypes: geometry(1), object(12)
memory usage: 43.8+ KB


In [14]:
#Dropped the 4 rows that has NaN values in lat/lon
#Showing .info to confirm that lon and lat are now floats and only 426 rows, 
#    with no non-null values.
racks = racks.dropna(subset=['lat'])
racks['lat'] = racks.lat.astype(float)
racks['lon'] = racks.lon.astype(float)
racks.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 426 entries, 0 to 429
Data columns (total 13 columns):
detail_loc    426 non-null object
quantity      426 non-null object
location      426 non-null object
domain        426 non-null object
idnumber      426 non-null object
objectid      426 non-null object
status        426 non-null object
lon           426 non-null float64
type          426 non-null object
data_colle    426 non-null object
lat           426 non-null float64
capacity      426 non-null object
geometry      426 non-null geometry
dtypes: float64(2), geometry(1), object(10)
memory usage: 46.6+ KB


In [7]:
racks.shape

(426, 13)

In [12]:
map_racks = folium.Map(
    location = [36.174465, -86.767960],    #Nashville coordinates
    zoom_start = 12
)

for row_index, row_values in racks.iterrows():
    loc = [row_values['lat'], row_values['lon']]
    pop = str(row_values['detail_loc'])
    marker = folium.Marker(
        location = loc, 
        popup = pop) 
    
    marker.add_to(map_racks)
map_racks.save('../maps/map_racks.html')   #Tell it the path to save the map to (make sure we have a maps folder)

#display our map
map_racks

In [11]:
culster_map_racks = folium.Map(
    location = [36.174465, -86.767960],    #Nashville coordinates
    zoom_start = 12
)

#create a marker cluster, and adds it to the map.
marker_cluster = MarkerCluster().add_to(culster_map_racks)

# inside the loop add each marker to the cluster
for row_index, row_values in racks.iterrows():
    loc = [row_values['lat'], row_values['lon']]
    pop = str(row_values['detail_loc'])
    marker = folium.Marker(
        location = loc, 
        popup = pop) 
    
    marker.add_to(marker_cluster)
    
culster_map_racks.save('../maps/culster_map_racks.html')   #Tell it the path to save the map to (make sure we have a maps folder)

#display our map
culster_map_racks